In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your_HuggingFace_API_Token"
os.environ["GEMINI_API_KEY"] = "Your_Gemini_API_Key"

In [159]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [160]:
video_id = "mBdVK4cqiFs"

try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    transcript = " ".join([chunk["text"] for chunk in transcript_list])
    print(transcript)
except TranscriptsDisabled:
    print("Transcript is disabled for this video")

in this video we're going to talk about nuclear fission unstable radioactive nuclei which occur naturally sometimes emits strong radiations especially in isotopes of uranium like uranium-238 or uranium-235 the disintegration converts the uranium isotope into stable lead and in the process emits energy in the range of one to five mega electron volts that's one to five million electron volts and even higher amounts of energy can be released by breaking unstable nuclei into even more parts in 1939 a german scientist named otto han discovered that breaking up the nucleus of uranium-235 into two parts emits 200 million times the energy of the neutron which triggered it he along with his two colleagues lisa meitner and fritz strassmann found that when the nucleus of uranium-235 is bombarded with neutrons it will absorb one of the neutrons this results in an unstable compound nucleus and the nucleus breaks into two equal parts this process is referred to as nuclear fission so a process in whi

Step 1b - Indexing (Text Splitting)

In [161]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [162]:
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embedding_function)

In [ ]:
vector_store.index_to_docstore_id

{0: 'e6630e97-8486-49d6-afb7-4d50913dc585',
 1: '15b32a18-a599-4e0b-bba9-8cdb1199b6b4',
 2: '42cab7e0-0dcf-4265-8243-678cfe214521',
 3: '4268f98f-9a02-48b9-9f50-874eb58c2e31'}

In [164]:
vector_store.get_by_ids(['4269a815-53d8-451e-bb98-ae249771d3c3'])

[]

Step 2 - Retrieval

In [165]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [166]:
#retriever.invoke("What is the main topic of the video?")

In [167]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

In [168]:
prompt = PromptTemplate(
    template="Answer the question based on the context below.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:",
    input_variables=["context", "question"]
)

In [169]:
question = "Is nuclear fission discussed in the video?If yes, what is the explanation provided?"
retrieved_docs = retriever.invoke(question)

In [170]:
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [171]:
final_prompt = prompt.invoke({"context":context_text, "question":question})

Step 4 - Generation

In [172]:
answer = llm.invoke(final_prompt)
print(answer)

content="Yes, nuclear fission is discussed in the video.\n\n**Explanation provided:**\nNuclear fission is defined as a process in which heavy nuclei are bombarded with neutrons and split into two equal masses, releasing enormous amounts of energy.\n\nThe process is explained as follows:\n1.  Unstable radioactive nuclei, such as uranium-235, can emit strong radiations.\n2.  When the nucleus of uranium-235 is bombarded with a neutron, it absorbs the neutron, forming a highly unstable compound nucleus, uranium-236.\n3.  This unstable uranium-236 nucleus then splits into two product nuclei: barium-141 and krypton-92.\n4.  In addition to these two nuclei, three neutrons are also emitted, along with a high amount of energy (denoted as 'q' in the chemical equation). For instance, breaking up uranium-235 emits 200 million times the energy of the neutron which triggered it.\n5.  The three neutrons emitted can then be absorbed by other uranium-235 nuclei, leading to a chain reaction that results

In [173]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [174]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [175]:
parallel_chain = RunnableParallel({
    'context' : retriever | RunnableLambda(format_docs),
    'question' : RunnablePassthrough()
})

In [176]:
parallel_chain.invoke('Who is Demis')

{'context': "neutrons in the process this happens because barium-141 has 141 neutrons and krypton 92 has 92 neutrons adding up to a total of 233 neutrons to balance the equation uranium-236 must emit three more neutrons the chemical equation of the nuclear fission reaction taking place can be represented by this equation so as you can see from the equation one neutron combines with uranium-235 forming an unstable compound nucleus uranium-236 this unstable nucleus splits into two stable nuclei barium-141 and krypton 92. in addition to these two nuclei three neutrons are also emitted as well as a high amount of energy denoted in the equation by q since uranium-236 emits three neutrons in the reaction these neutrons can be absorbed by other uranium-235 nuclei resulting in a chain reaction this leads to exponentially high amounts of energy being released by a single reaction so in this video we've covered that nuclear fission reactions result in massive amounts of energy being released now

In [177]:
parser = StrOutputParser()

In [178]:
main_chain = parallel_chain | prompt | llm | parser

In [179]:
main_chain.invoke("summarize the content of the video in a sentence")

'This video explains nuclear fission as the process where heavy nuclei, such as uranium-235, are bombarded by neutrons, causing them to split into two parts, releasing massive amounts of energy and additional neutrons that can trigger a chain reaction.'